# Description

In [1]:
# TODO(Grisha): unit test the notebook.

The notebook loads prod system run output for debugging purposes. E.g., loads DAG output for a specific node / bar timestamp to check the data.

# Imports

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import logging
import os

import pandas as pd

import core.config as cconfig
import core.finance as cofinanc
import dataflow.core as dtfcore
import helpers.hdbg as hdbg
import helpers.henv as henv
import helpers.hprint as hprint
import reconciliation as reconcil

/app/amp/core/signal_processing/decorrelation.py:12: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [4]:
hdbg.init_logger(verbosity=logging.INFO)

_LOG = logging.getLogger(__name__)

_LOG.info("%s", henv.get_system_signature()[0])

hprint.config_notebook()

INFO: > cmd='/venv/lib/python3.8/site-packages/ipykernel_launcher.py -f /home/.local/share/jupyter/runtime/kernel-74dd5ddb-861c-4074-84e5-6e7d73b4d3d6.json'
-----------------------------------------------------------------------------
This code is not in sync with the container:
code_version='1.4.2' != container_version='1.4.0'
-----------------------------------------------------------------------------
You need to:
- merge origin/master into your branch with `invoke git_merge_master`
- pull the latest container with `invoke docker_pull`
INFO  # Git
  branch_name='CmTask4723_C5b_failure'
  hash='6d86ca9b0'
  # Last commits:
    *   6d86ca9b0 DanilYachmenev Merge branch 'master' into CmTask4723_C5b_failure                 (18 minutes ago) Thu Jul 13 14:04:50 2023  (HEAD -> CmTask4723_C5b_failure, origin/CmTask4723_C5b_failure)
    |\  
    | * fe2db79c9 Samarth KaPatel CmTask4744 System reconciliation gdoc to md (#4764)               (82 minutes ago) Thu Jul 13 13:01:09 2023  (origin/m

# Build the config

In [5]:
# Get config from env when running the notebook via the `run_notebook.py` script, e.g.,
# in the system reconciliation flow.
config = cconfig.get_config_from_env()
if config:
    _LOG.info("Using config from env vars")
else:
    _LOG.info("Using hardwired config")
    # Specify the config directly when running the notebook manually.
    # Below is just an example.
    config_dict = {
        "dst_root_dir": "/shared_data/ecs/preprod/system_reconciliation",
        "dag_builder_name": "C5b",
        "run_mode": "paper_trading",
        "start_timestamp_as_str": "20230713_131000",
        "end_timestamp_as_str": "20230714_130500",
        "mode": "scheduled",
        "node_name": "predict.9.process_forecasts",
        "bar_timestamp": pd.Timestamp("2023-07-13 10:20:00-04:00"),
        "asset_id": 2484635488,
        "columns": ["close", "feature"],
    }
    config = cconfig.Config.from_dict(config_dict)
print(config)

INFO  Using hardwired config
dst_root_dir: /shared_data/ecs/preprod/system_reconciliation
dag_builder_name: C5b
run_mode: paper_trading
start_timestamp_as_str: 20230713_131000
end_timestamp_as_str: 20230714_130500
mode: scheduled
node_name: predict.9.process_forecasts
bar_timestamp: 2023-07-13 10:20:00-04:00
asset_id: 2484635488
columns: ['close', 'feature']


# Specify data to load

In [6]:
target_dir = reconcil.get_target_dir(
    config["dst_root_dir"],
    config["dag_builder_name"],
    config["run_mode"],
    config["start_timestamp_as_str"],
    config["end_timestamp_as_str"],
)
system_log_dir = reconcil.get_prod_system_log_dir(config["mode"])
system_log_dir = os.path.join(target_dir, system_log_dir)
_LOG.info("system_log_dir=%s", system_log_dir)

INFO  dst_root_dir='/shared_data/ecs/preprod/system_reconciliation', dag_builder_name='C5b', run_mode='paper_trading', start_timestamp_as_str='20230713_131000', end_timestamp_as_str='20230714_130500'
INFO  target_dir='/shared_data/ecs/preprod/system_reconciliation/C5b/paper_trading/20230713_131000.20230714_130500'
INFO  system_log_dir='system_log_dir.scheduled'
INFO  system_log_dir=/shared_data/ecs/preprod/system_reconciliation/C5b/paper_trading/20230713_131000.20230714_130500/system_log_dir.scheduled


In [7]:
dag_data_dir = reconcil.get_data_type_system_log_path(system_log_dir, "dag_data")
_LOG.info("dag_data_dir=%s", dag_data_dir)

INFO  dag_data_dir=/shared_data/ecs/preprod/system_reconciliation/C5b/paper_trading/20230713_131000.20230714_130500/system_log_dir.scheduled/dag/node_io/node_io.data


In [8]:
# Get DAG node names.
dag_node_names = dtfcore.get_dag_node_names(dag_data_dir)
dag_node_names

['predict.0.read_data',
 'predict.1.resample',
 'predict.2.compute_vol',
 'predict.3.compute_ret_0',
 'predict.4.adjust_rets',
 'predict.5.compress_rets',
 'predict.6.demean',
 'predict.7.compress_residuals',
 'predict.8.generate_feature',
 'predict.9.process_forecasts']

In [9]:
# Get timestamps for the specified DAG node.
dag_node_timestamps = dtfcore.get_dag_node_timestamps(
    dag_data_dir, config["node_name"], as_timestamp=True
)
_LOG.info(
    "First timestamp='%s'/ Last timestamp='%s'",
    dag_node_timestamps[0][0],
    dag_node_timestamps[-1][0],
)

INFO  First timestamp='2023-07-13 09:10:00-04:00'/ Last timestamp='2023-07-13 10:20:00-04:00'


# Load data

In [10]:
# Load data for a given DAG node and a bar timestmap.
node_df = dtfcore.get_dag_node_output(
    dag_data_dir, config["node_name"], config["bar_timestamp"]
)
node_df.tail(3)

feature                                                                                                                                                                                                                                                                         feature_raw.c                                                                                                                                                                                                                                                                         feature_raw                                                                                                                                                                                                                                                                         vwap.ret_0.vol_adj.c                                                                                                                           \
                          1030828978 1464553467 1467591036 1528092593 1776791608 1891737434 1966583502 2099673105 2237530510 2384892553 2425308589 2476706208 2484635488 2540896331 2601760471 2683705052 3065029174 3401245610 4516629366 5115052901 5118394986 6051632686 8717633868 8968126878 9872743573    1030828978 1464553467 1467591036 1528092593 1776791608 1891737434 1966583502 2099673105 2237530510 2384892553 2425308589 2476706208 2484635488 2540896331 2601760471 2683705052 3065029174 3401245610 4516629366 5115052901 5118394986 6051632686 8717633868 8968126878 9872743573  1030828978 1464553467 1467591036 1528092593 1776791608 1891737434 1966583502 2099673105 2237530510 2384892553 2425308589 2476706208 2484635488 2540896331 2601760471 2683705052 3065029174 3401245610 4516629366 5115052901 5118394986 6051632686 8717633868 8968126878 9872743573           1030828978 1464553467 1467591036 1528092593 1776791608 1891737434 1966583502 2099673105 2237530510 2384892553 2425308589 2476706208   
end_timestamp                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
2023-07-13 10:10:00-04:00  -0.108552  -0.135369   1.321589   1.266724  -0.971728   0.607414  -0.593955   0.545474  -0.339026  -3.060387   0.712132   0.121385  -0.683924  -0.590520  -0.606037   0.013112  -0.209048   0.982116  -0.355445   1.090283  -0.351341   0.528553   0.908852  -0.098481   0.829710      0.108552   0.135369  -1.321589  -1.266724   0.971728  -0.607414   0.593955  -0.545474   0.339026   3.060387  -0.712132  -0.121385   0.683924   0.590520   0.606037  -0.013112   0.209048  -0.982116   0.355445  -1.090283   0.351341  -0.528553  -0.908852   0.098481  -0.829710    0.108578   0.135421  -1.373096  -1.311815   0.991550  -0.612149   0.598379  -0.548894   0.339841   4.033573  -0.719803  -0.121422   0.690708   0.594867   0.610739  -0.013112   0.209238  -1.002598   0.356385  -1.118555   0.352249  -0.531662  -0.924995   0.098501  -0.841927            -1.818123  -1.791280  -3.299798  -3.238517  -0.935151  -2.538850  -1.328323  -2.475595  -1.586860   2.106872  -2.646504  -2.048124   
2023-07-13 10:15:00-04:00   0

In [11]:
# Filter by specified asset and display only relevant columns.
slice_df = cofinanc.get_asset_slice(node_df, config["asset_id"])[
    config["columns"]
]
slice_df.tail(3)

,close,feature
end_timestamp,,
2023-07-13 10:10:00-04:00,0.0808,-0.683924
2023-07-13 10:15:00-04:00,0.0808,-0.286388
2023-07-13 10:20:00-04:00,0.0808,0.099717
